In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('recommender_system').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
data = spark.read.csv('data/movielens_ratings.csv', inferSchema=True, header=True)
print(data.count(), len(data.columns))
data.printSchema()
data.show(5)
data.head(1)

1501 3
root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



[Row(movieId=2, rating=3.0, userId=0)]

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
train_data, test_data = data.randomSplit([0.8, 0.2])
print(train_data.count())
print(test_data.count())

1196
305


In [10]:
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating'
)
model = als.fit(train_data)
predictions = model.transform(test_data)
predictions.orderBy('movieId').show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   1.0|    20|  1.3726354|
|      0|   3.0|    28|  3.2430103|
|      0|   1.0|     3|-0.22144218|
|      0|   1.0|    22| 0.21939962|
|      1|   1.0|     7|  1.0775502|
|      1|   1.0|    14| 0.56598014|
|      1|   1.0|     5|   2.331779|
|      2|   4.0|     8|  1.5524069|
|      2|   2.0|     7|  1.3128681|
|      2|   3.0|     9| -3.6783395|
|      2|   1.0|    15|  1.6849923|
|      2|   4.0|    28|  5.8614545|
|      3|   1.0|    13|  1.7975401|
|      3|   1.0|     9|-0.21313845|
|      4|   2.0|     8|  1.4898245|
|      4|   1.0|    23|-0.06936586|
|      4|   2.0|    13|   2.372957|
|      4|   1.0|    14|   1.117142|
|      5|   2.0|    15|   1.527002|
|      5|   1.0|     9|  1.7599862|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print(rmse) # root mean squared error

1.8879953469718325


In [15]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId','userId'])
# single_user.show()
recommendations = model.transform(single_user)
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     18|    11| 3.3179247|
|     11|    11|  3.150256|
|     94|    11| 3.0081873|
|     27|    11| 2.3751745|
|     76|    11| 2.0040026|
|     75|    11| 1.9237936|
|     45|    11| 0.7069759|
|     78|    11|0.66476303|
|     51|    11| 0.3098364|
|     21|    11|-0.5927002|
|     35|    11|-1.5646162|
|     37|    11|-1.9340689|
+-------+------+----------+



In [ ]:
# fundamental question - what about cold start? no history